In [41]:
import json
import pandas as pd
import re 
import glob 
import copy 
import os 
import nltk
import unicodedata 
from collections import Counter
from nltk.collocations import *
import numpy as np 

In [2]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
import spacy

In [3]:
# !pip install -U pip setuptools wheel
# !pip install -U spacy
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('omw-1.4')
#!python -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")
import en_core_web_sm
nlp = en_core_web_sm.load()
from bs4 import BeautifulSoup
from html import unescape

In [4]:
%%time 
### Creating reviews dataframe 
data_file = open("yelp_academic_dataset_review.json",encoding='utf-8')
data = []
for line in data_file:
    data.append(json.loads(line))
reviewsdf = pd.DataFrame(data)
data_file.close()

Wall time: 6min 47s


In [5]:
### Creataing businesses dataframe 
data_file = open("yelp_academic_dataset_business.json",encoding='utf-8')
data = []
for line in data_file:
    data.append(json.loads(line))
businessdf = pd.DataFrame(data)
data_file.close()

## For now, use smaller version of reviewsdf just to make sure everything works

In [6]:
reviewsdf = reviewsdf.head(10000)

### Generate hashtags from reviews df

In [7]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [8]:
#Adjust mindf parameter
mindf = int(len(reviewsdf) * .001)
tfidf = TfidfVectorizer(sublinear_tf=True,
               analyzer='word',
#                max_features=2000,
               tokenizer=word_tokenize,
               stop_words=nltk.corpus.stopwords.words("english"),min_df=5)

In [9]:
text = list(reviewsdf['text'])
text = [re.sub(r'[^A-Za-z]+'," ",document) for document in text]


In [10]:
%%time
hashtagdf = pd.DataFrame(tfidf.fit_transform(text).toarray())

C:\Users\mrpie\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  warnings.warn(


Wall time: 4.06 s


In [11]:
hashtagdf.columns = tfidf.get_feature_names()

C:\Users\mrpie\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [12]:
hashtagdfsum = hashtagdf.sum()
tfidf_scores = [(i,hashtagdfsum[i]) for i in hashtagdfsum.index]
tfidf_scores.sort(key=lambda x : x[1],reverse=True)

In [13]:
##REMOVE?
lower_threshold = int(len(tfidf_scores) * .08)
upper_threshold = int(len(tfidf_scores) * .60)
scores = tfidf_scores[lower_threshold:]
tfidf_dict = {item[0] : item[1] for item in scores}

In [14]:
#Maybe add ability to get N most useful tags 
def generate_tags(text):
    tags = set()
    for s in text.split(' '):
        if s in tfidf_dict:
            tags.add(s)
    return list(tags)[:5]

In [15]:
reviewsdf['tags'] = reviewsdf['text'].apply(generate_tags)

In [16]:
reviewsdf.head(5)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,tags
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11,"[beginning, multiple, takes, decide, opt]"
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5.0,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18,"[dropping, online, classes, passion, compares]"
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3.0,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30,"[relaxed, rolled, grape, casual, choices]"
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03,"[changed, kinds, deter, curry, lamb]"
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4.0,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15,"[limited, added, tomato, draft, jelly]"


In [17]:
# trigram_measures = nltk.collocations.TrigramAssocMeasures()


In [18]:
# finder = BigramCollocationFinder.from_words()
# finder1 = finder.apply_freq_filter(1)
# finder.nbest(bigram_measures.pmi, 1500)
# #Maybe add bigrams to each string, so they end upo in freq dict
# # list(bigrams)

### Hashtag generation above needs to be improved, just temporarlily using it to continue with other parts

# Create graph data

1 ) Create  weighted edges between business and associated hashtags (edge weight is number of reviews that contained particular hashtag?)

2) Create weighted edges between hashtags that occur in the same business (or review ?) weight = num co coccurances in reviews?

Drop unecessary columns from reviewsdf 

In [212]:
reviewsdf = reviewsdf[['review_id','text','business_id','tags']]
reviewsdf

,review_id,text,business_id,tags
0,KU_O5udG6zpxOg-VcAEodg,"If you decide to eat here, just be aware it is...",XQfwVwDr-v0ZS3_CbbE5Xw,"[beginning, multiple, takes, decide, opt]"
1,BiTunyQ73aT9WBnpR9DZGw,I've taken a lot of spin classes over the year...,7ATYjTIgM3jUlt4UM3IypQ,"[dropping, online, classes, passion, compares]"
2,saUsX_uimxRlCVr67Z4Jig,Family diner. Had the buffet. Eclectic assortm...,YjUWPpI6HXG530lwP-fb2A,"[relaxed, rolled, grape, casual, choices]"
3,AqPFMleE6RsU23_auESxiA,"Wow! Yummy, different, delicious. Our favo...",kxX2SOes4o-D3ZQBkiMRfA,"[changed, kinds, deter, curry, lamb]"
4,Sx8TMOWLNuJBWer-0pcmoA,Cute interior and owner (?) gave us tour of up...,e4Vwtrqf-wpJfwesgvdgxQ,"[limited, added, tomato, draft, jelly]"
...,...,...,...,...
9995,ZcBtCA9jGhLfakf1jJ2BAg,Excellent food and service. The place is funct...,1-z7wd860Rii4kbEMCT8DA,"[welcoming, funky, choices, environment]"
9996,UIkEO-10J6Y99IhRqUflvg,Just about to get tucked into a meatloaf that ...,EpREWeEpmR8f1qLHzzF0AA,"[meatloaf, smelling, tucked]"
9997,S-NQM3Axcg8JS3MXHUIvyw,Outstanding customer service! And my car is dr...,dvidzWEPgTQPeBc8CUV2OQ,"[taken, experienced, needing, talk, shops]"
9998,ME79YrEhm2xe4IQy_0zkGw,I and my husband went here for Dinner one day ...,2XYPFRm7teCUr3eGsB2-qw,"[environment, vegetarian, lit, noodles, closer]"


In [228]:
###Create graph between business and associated hashtags 
from collections import defaultdict
businessAndTagGraph = defaultdict(lambda: defaultdict(int))
tagsGraph = defaultdict(lambda: defaultdict(int))

In [229]:
%%time
for review in reviewsdf.itertuples():
    businessId = review[3]
    tags = review[4]
    for i in range(len(tags)): 
        #Update Edge weights (number of times hashtag occurs in review)
        currTag = tags[i]
        businessAndTagGraph[businessId][currTag] += 1
        #Create edges and weights for tags in same review 
        for j in range(len(tags)):
            if i != j:
                 tagsGraph[currTag][tags[j]] += 1


Wall time: 140 ms


#### Convert business and tags graph into dataframe/csv format

In [230]:
#businessAndTagsdf = pd.DataFrame(columns=['business_id','hashtag','count'])
idcol = []
hashtagcol = []
countcol = []
ignore = set(['index_names','column_names','index','columns','data'])
for businessid in businessAndTagGraph.keys():
    if businessid in ignore:
        continue
    for vals in businessAndTagGraph[businessid].items():
        idcol.append(businessid)
        hashtagcol.append(vals[0])
        countcol.append(vals[1])
        #businessAndTagsdf.append({'business_id':businessid,'hashtag':vals[0],count:vals[1]})


In [231]:
businessAndTagsdf = pd.DataFrame(columns = ['business_id','hashtag','count'])
businessAndTagsdf['business_id'] = idcol
businessAndTagsdf['hashtag'] = hashtagcol
businessAndTagsdf['count'] = countcol

In [232]:
#Resulting df 
businessdf = businessdf[['business_id','name','address','state','latitude','longitude','stars','categories']]

In [245]:
businessAndTagsdf = businessAndTagsdf.merge(businessdf,on='business_id')
businessAndTagsdf

,business_id,hashtag,count,name_x,address_x,state_x,latitude_x,longitude_x,stars_x,categories_x,...,longitude_y,stars_y,categories_y,name,address,state,latitude,longitude,stars,categories
0,XQfwVwDr-v0ZS3_CbbE5Xw,beginning,1,Turning Point of North Wales,1460 Bethlehem Pike,PA,40.210196,-75.223639,3.0,"Restaurants, Breakfast & Brunch, Food, Juice B...",...,-75.223639,3.0,"Restaurants, Breakfast & Brunch, Food, Juice B...",Turning Point of North Wales,1460 Bethlehem Pike,PA,40.210196,-75.223639,3.0,"Restaurants, Breakfast & Brunch, Food, Juice B..."
1,XQfwVwDr-v0ZS3_CbbE5Xw,multiple,1,Turning Point of North Wales,1460 Bethlehem Pike,PA,40.210196,-75.223639,3.0,"Restaurants, Breakfast & Brunch, Food, Juice B...",...,-75.223639,3.0,"Restaurants, Breakfast & Brunch, Food, Juice B...",Turning Point of North Wales,1460 Bethlehem Pike,PA,40.210196,-75.223639,3.0,"Restaurants, Breakfast & Brunch, Food, Juice B..."
2,XQfwVwDr-v0ZS3_CbbE5Xw,takes,1,Turning Point of North Wales,1460 Bethlehem Pike,PA,40.210196,-75.223639,3.0,"Restaurants, Breakfast & Brunch, Food, Juice B...",...,-75.223639,3.0,"Restaurants, Breakfast & Brunch, Food, Juice B...",Turning Point of North Wales,1460 Bethlehem Pike,PA,40.210196,-75.223639,3.0,"Restaurants, Breakfast & Brunch, Food, Juice B..."
3,XQfwVwDr-v0ZS3_CbbE5Xw,decide,1,Turning Point of North Wales,1460 Bethlehem Pike,PA,40.210196,-75.223639,3.0,"Restaurants, Breakfast & Brunch, Food, Juice B...",...,-75.223639,3.0,"Restaurants, Breakfast & Brunch, Food, Juice B...",Turning Point of North Wales,1460 Bethlehem Pike,PA,40.210196,-75.223639,3.0,"Restaurants, Breakfast & Brunch, Food, Juice B..."
4,XQfwVwDr-v0ZS3_CbbE5Xw,opt,1,Turning Point of North Wales,1460 Bethlehem Pike,PA,40.210196,-75.223639,3.0,"Restaurants, Breakfast & Brunch, Food, Juice B...",...,-75.223639,3.0,"Restaurants, Breakfast & Brunch, Food, Juice B...",Turning Point of North Wales,1460 Bethlehem Pike,PA,40.210196,-75.223639,3.0,"Restaurants, Breakfast & Brunch, Food, Juice B..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39899,K_L_w0Hv47TLlpNJ2R14zA,using,1,South Reno Athletic Club,9393 Gateway Dr,NV,39.445329,-119.766671,4.5,"Gyms, Active Life, Fitness & Instruction",...,-119.766671,4.5,"Gyms, Active Life, Fitness & Instruction",South Reno Athletic Club,9393 Gateway Dr,NV,39.445329,-119.766671,4.5,"Gyms, Active Life, Fitness & Instruction"
39900,K_L_w0Hv47TLlpNJ2R14zA,training,1,South Reno Athletic Club,9393 Gateway Dr,NV,39.445329,-119.766671,4.5,"Gyms, Active Life, Fitness & Instruction",...,-119.766671,4.5,"Gyms, Active Life, Fitness & Instruction",South Reno Athletic Club,9393 Gateway Dr,NV,39.445329,-119.766671,4.5,"Gyms, Active Life, Fitness & Instruction"
39901,K_L_w0Hv47TLlpNJ2R14zA,larger,1,South Reno Athletic Club,9393 Gateway Dr,NV,39.445329,-119.766671,4.5,"Gyms, Active Life, Fitness & Instruction",...,-119.766671,4.5,"Gyms, Active Life, Fitness & Instruction",South Reno Athletic Club,9393 Gateway Dr,NV,39.445329,-119.766671,4.5,"Gyms, Active Life, Fitness & Instruction"
39902,K_L_w0Hv47TLlpNJ2R14zA,sheer,1,South Reno Athletic Club,9393 Gateway Dr,NV,39.445329,-119.766671,4.5,"Gyms, Active Life, Fitness & Instruction",...,-119.766671,4.5,"Gyms, Active Life, Fitness & Instruction",South Reno Athletic Club,9393 Gateway Dr,NV,39.445329,-119.766671,4.5,"Gyms, Active Life, Fitness & Instruction"


##### Convert tags graph to df/csv format

In [236]:
tag1 = []
tag2 = []
sharedOccurances = []
for tag in tagsGraph.keys():
    for vals in tagsGraph[tag].items():
        tag1.append(tag)
        tag2.append(vals[0])
        sharedOccurances.append(vals[1])

In [240]:
tagsDf = pd.DataFrame(columns = ['tag1','tag2','sharedOccurances'])
tagsDf['tag1'] = tag1
tagsDf['tag2'] = tag2
tagsDf['sharedOccurances']= sharedOccurances

In [243]:
tagsDf

,tag1,tag2,sharedOccurances
0,beginning,multiple,1
1,beginning,takes,1
2,beginning,decide,1
3,beginning,opt,1
4,beginning,studio,1
...,...,...,...
143519,morsel,nearby,1
143520,morsel,tend,1
143521,morsel,specialty,1
143522,morsel,freshly,1


#### Save graphs to csv

In [ ]:
businessAndTagsdf.to_csv('businessAndTags.csv')
tagsDf.to_csv('tagsGraph.csv')